In [105]:
!pip3 -q install pandas numpy langchain langsmith langgraph langchain_openai langchain-community langchainhub langchain-openai langchain-chroma bs4


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [113]:
!pip3 -q install torch


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [115]:
!pip3 install tranformers[sentencepiece]

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement tranformers[sentencepiece] (from versions: none)
ERROR: No matching distribution found for tranformers[sentencepiece]

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [139]:
!pip3 install fastembed

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for fastembed from https://files.pythonhosted.org/packages/d8/bb/fa7620fd49aa073c16d410e6cbdb4f992b0c52135f4f12ca0b1dbff47287/fastembed-0.2.7-py3-none-any.whl.metadata
  Obtaining dependency information for huggingface-hub<0.21,>=0.20 from https://files.pythonhosted.org/packages/28/03/7d3c7153113ec59cfb31e3b8ee773f5f420a0dd7d26d40442542b96675c3/huggingface_hub-0.20.3-py3-none-any.whl.metadata
  Obtaining dependency information for loguru<0.8.0,>=0.7.2 from https://files.pythonhosted.org/packages/03/0a/4f6fed21aa246c6b49b561ca55facacc2a44b87d65b8b92362a8e99ba202/loguru-0.7.2-py3-none-any.whl.metadata
  Obtaining dependency information for onnx<2.0.0,>=1.15.0 from https://files.pythonhosted.org/packages/e8/e3/2eba2167d36a845af16255fe9c2a0a22a7034f3765109790cab91038c167/onnx-1.16.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency inform

In [142]:
!pip3 install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/b5/be/23bfb89fdeacf133dffd905787c620740d89d3fd35ffcf98bdd31f9348f7/faiss_cpu-1.8.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 15.6 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip3 install --upgrade pip


In [136]:
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

In [137]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    "https://blog.langchain.dev/langchain-v0-1-0/"
)

documents = loader.load()

In [135]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 50
)

documents = text_splitter.split_documents(documents)

In [140]:
embeddings = FastEmbedEmbeddings()

/home/debian/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.07it/s]


In [143]:
from langchain_community.vectorstores import FAISS

vector_store = FAISS.from_documents(documents, embeddings)

In [144]:
retriever = vector_store.as_retriever()

In [145]:
retrieved_documents = retriever.invoke("Why did they change to version 0.1.0?")

In [147]:
from langchain import hub

retrieval_qa_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

In [148]:
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [149]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

primary_qa_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | primary_qa_llm, "context": itemgetter("context")}
)

In [124]:
llm = HuggingFaceEndpoint(
    endpoint_url="<get mistral api key from https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2>",
    task="text_generation",
    max_new_token=6096,
    huggingfacehub_api_token="<get huggingface token>"
)

WARNING! max_new_token is not default parameter.
                    max_new_token was transferred to model_kwargs.
                    Please make sure that max_new_token is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/debian/.cache/huggingface/token
Login successful


In [150]:
question = "What are the major changes in v0.1.0?"

result = retrieval_augmented_qa_chain.invoke({"question" : question})

print(result["response"])

The major changes in v0.1.0 include separating out langchain-core and partner packages, introducing a new versioning standard, improving stability, and focusing on core functionality and documentation.


##https://www.youtube.com/watch?v=Anr1br0lLz8&list=PLrSHiQgy4VjGjrLbBc9grlnXgkOeP4tQ-&index=2
following above playlist

AI Markerspace channel